In [1]:
!pip install transformers==3.5.0
!pip install torch==1.7.0
!pip install rouge-score

from transformers import PegasusTokenizer,PegasusForConditionalGeneration
from rouge_score import rouge_scorer
import torch
import os
from tqdm import tqdm
import pandas as pd
import nltk
nltk.download('punkt')

     |████████████████████████████████| 1.3MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 34.4MB/s 
     |████████████████████████████████| 890kB 44.7MB/s 
     |████████████████████████████████| 2.9MB 37.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5ccbd950b3427b236eff800830340442cf52cd6a2f8750f979097cbfb65107ff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:

check_path = "/content/AMICorpusXML/data/ami-summary/abstractive/ES2004a.abssumm.txt"
from pathlib import Path

my_file = Path(check_path)
if not my_file.is_file():
    if not Path("/content/AMICorpusXML/data").is_dir():
        ! git clone https://github.com/gcunhase/AMICorpusXML
    ! python /content/AMICorpusXML/main_obtain_meeting2summary_data.py --summary_type abstractive
else:
    print("Data already prepared... Importing\n")

Cloning into 'AMICorpusXML'...
remote: Enumerating objects: 9453, done.
remote: Counting objects: 100% (9453/9453), done.
remote: Compressing objects: 100% (9271/9271), done.
remote: Total 9664 (delta 196), reused 9426 (delta 170), pack-reused 211
Receiving objects: 100% (9664/9664), 9.67 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (224/224), done.
Extracting speaker transcripts to: data/ami-transcripts-speaker/
Transcripts: 687

Saving transcripts in: data/ami-transcripts/
Number of meetings: 171

Obtaining abstractive summaries to: data/ami-summary/
Make .story files
Make .story files


In [8]:
# taking same split to make easy comparision
# data split knowledge: http://groups.inf.ed.ac.uk/ami/corpus/datasets.shtml
# 5 x 4 = 20 
test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(',')

val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
            ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
            TS3011, TS3012""".split(",")

story = []
story_directory = r'/content/AMICorpusXML/data/ami-transcripts-stories/abstractive'
for filename in os.listdir(story_directory):
    if filename.endswith(".story"):
        for each in test:
            if each.strip() in str(filename):
                story.append(filename)
     
summary = []
sum_directory = r'/content/AMICorpusXML/data/ami-summary/abstractive'
for filename in os.listdir(sum_directory):
    if filename.endswith(".txt"):
        for each in test:
            if each.strip() in str(filename):
                summary.append(filename)

def prepare_model(model_name = 'google/pegasus-xsum'):
    print("----------------- Preparing Model -----------------")
    
    torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
    print("----------------- Model Prepared -----------------")
    return tokenizer, model, torch_device

def test_model(src_text, tokenizer, model):
    print("\nNew Testing started...")
    batch = tokenizer.prepare_seq2seq_batch(src_text, padding='max_length').to(torch_device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

def data_from_file(f1,f2):
    with open(f1, 'r') as file:
        data1 = file.read().replace('\n', '')
    with open(f2, 'r') as file:
        data2 = file.read().replace('\n', '')
    return data1, data2

df = pd.DataFrame()
st, sums_pred, sums_ann = [], [], []

# remove later
# flag = 0

tokenizer, model, torch_device = prepare_model()
for each in tqdm(story):
    a = each.split('.')[0]
    story_file = story_directory+'/'+each
    summ_file = sum_directory+'/'+a+".abssumm.txt"
    story_file, summ_file = data_from_file(story_file, summ_file)
    st.append(story_file)
    src_text = [story_file.replace('\n',' ')]
    result = test_model(src_text, tokenizer, model)
    sums_pred.append(result[0])
    sums_ann.append(summ_file)

    # flag+=1
    # if flag >=2: break

df['stories'] = st
df['summaries'] = sums_pred
df['Summary annotated'] = sums_ann
df.to_csv('summary_df.csv')
display(df.head())

  0%|          | 0/20 [00:00<?, ?it/s]


New Testing started...


  5%|▌         | 1/20 [00:21<06:41, 21.15s/it]


New Testing started...


 10%|█         | 2/20 [00:43<06:24, 21.37s/it]

...


New Testing started...


 95%|█████████▌| 19/20 [08:19<00:25, 25.96s/it]


New Testing started...


100%|██████████| 20/20 [08:48<00:00, 26.42s/it]


,stories,summaries,Summary annotated
0,Hello. Hi. Yeah. It's too beautiful. A minute ...,"Janus, welcome back to the functional design m...",When this functional design meeting opens the ...
1,Well hi everyone again. Um like before we uh I...,This is the meeting where we discuss the desig...,The project manager opens the meeting by going...
2,"Wouldn't wanna be Project Manager. Uh, what we...",Let's start with the presentation.,The project manager opened the meeting and wen...
3,"Okay. Yeah, my name is Francina. And I'm uh an...",In our series of letters from Australian stude...,The meeting opens with the group doing introdu...
4,"Okay, is everybody ready? Mm-hmm. Um I take it...",Mm-hmm.,The meeting begins and the marketing expert st...


In [9]:
# R1 and R2 for a sample summary 
a = df["stories"][0]
b = df["summaries"][0]
c = df["Summary annotated"][0]

def rogue_sc(b, a):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(b,a)
    return scores

scores = rogue_sc(b,a)
print("Word Capture: ", scores['rouge1'][1])
scores = rogue_sc(b,c)
print("ROUGE-1 Score: ", scores['rouge1'][1])
print("ROUGE-2 Score: ", scores['rouge2'][1])

Word Capture:  1.0
ROUGE-1 Score:  0.625
ROUGE-2 Score:  0.2857142857142857


In [11]:
import numpy as np

rouge1 = []
rouge2 = []
word_capt = []
for index, row in df.iterrows():
    rouge1.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge1'][1])
    rouge2.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge2'][1])
    word_capt.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge1'][1])
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

Average Rouge1 Score:  35.14517243762526
Average Rouge2 Score:  9.775370970492922
